In [1]:
# import argparse
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
# from torch_geometric.logging import init_wandb, log
# from torch_geometric.nn import MLP, GINConv, global_add_pool

from models.GraphESN import GESNModel

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
path = osp.join('dataset', 'TU')
dataset = TUDataset(path, name='MUTAG').shuffle()
batch_size = 64

train_dataset = dataset[len(dataset) // 10:]
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)

test_dataset = dataset[:len(dataset) // 10]
test_loader = DataLoader(test_dataset, batch_size)

In [4]:
train_dataset[0]

Data(edge_index=[2, 44], x=[21, 7], edge_attr=[44, 4], y=[1])

In [4]:
input_size = train_dataset[0].x.shape[1]

model = GESNModel(input_size, 500, 1, 0.9, 0.9, 0.9, 1., False).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Penso che GESNModel sia un dynamic Graph ESN

In [10]:
model(train_dataset[0].x.to(device), train_dataset[0].edge_index.to(device), train_dataset[0].edge_attr.to(device))

KeyboardInterrupt: 